# **Handling Duplicate Data**

In [ ]:
import pandas as pd
import numpy as np

data = {
    'EmpID': [101, 102, 103, 101, 104, 105, 102, 106, 107, 107],
    'Name': ['Ali', 'Sara', 'Ahmed', 'Ali', 'Zara', 'Usman', 'Sara', 'Hina', 'Bilal', 'Bilal'],
    'Dept': ['IT', 'HR', 'IT', 'IT', 'Finance', 'HR', 'HR', 'IT', 'HR', 'HR'],
    'Email': [
        'ali@corp.com',
        'sara@corp.com',
        'ahmed@corp.com',
        'ali@corp.com',          # duplicate email
        'zara@corp.com',
        'usman@corp.com',
        'sara@corp.com',         # duplicate email
        'hina@corp.com',
        'bilal@corp.com',
        'bilal@corp.com'         # duplicate email
    ],
    'Salary': [50000, 60000, 55000, 50000, 65000, 58000, 60000, 52000, 57000, 57000],
    'Bonus': [5000, 6000, 5500, 5000, 7000, 5800, None, 5200, 6000, 6000],
    'JoinDate': pd.to_datetime([
        '2022-01-01',
        '2021-06-15',
        '2022-03-20',
        '2022-01-01',
        '2020-09-10',
        '2021-11-05',
        '2021-06-15',
        '2022-07-01',
        '2021-04-18',
        '2021-04-18'
    ]),
    'UpdatedAt': pd.to_datetime([
        '2024-01-10',
        '2024-01-12',
        '2024-01-11',
        '2024-02-01',   # later update for Ali
        '2024-01-15',
        '2024-01-18',
        '2024-02-10',   # later update for Sara
        '2024-01-20',
        '2024-01-25',
        '2024-02-05'    # later update for Bilal
    ])
}

df = pd.DataFrame(data)

# Introduce duplicate index deliberately
df.index = [0, 1, 2, 0, 4, 5, 1, 7, 8, 8]

df


---
# **1. Detecting Duplicate Rows**
### **Check if ANY Duplicates Exist**

In [ ]:
df.duplicated().any()

### **Identify Duplicate Rows**

In [ ]:
df[df.duplicated()]

### **Count Duplicate Rows**


In [ ]:
df.duplicated().sum()

### **Detect Duplicates Based On Specific Columns**


In [ ]:
df.duplicated(subset=['Email'])
df['Email'].duplicated()

### **Get Rows Containing Duplicates Based on Specific Columns `subset=['ColName']`**

In [ ]:
df[df['Email'].duplicated()]
df[df.duplicated(subset=['Email'])]
df[df.duplicated(subset=['Name', 'Dept'])]


---
# **2. Removing Duplicate Rows**
### **Remove Duplicates (keep first)**

In [ ]:
df.drop_duplicates()

### **Remove Duplicates (keep last)**

In [ ]:
df.drop_duplicates(keep='last')

### **Remove All Duplicates (keep none)**

In [ ]:
df.drop_duplicates(keep=False)

### **Removing Rows Containing Duplicates Based on Specific Columns `subset=['ColName']`**

In [ ]:
df.drop_duplicates(subset=['Email'])

---
# **3. Handling Duplicate Indexes**
### **Detect Duplicate Indexes**

In [ ]:
df.index.duplicated()

### **Get Rows Containing Duplicate Indexes**


In [ ]:
df[df.index.duplicated()]

### **Remove Duplicate Indexes (Keep First)**


In [ ]:
df=df[~df.index.duplicated()]
df

### **Reset Index (most common fix)**


In [ ]:
df.reset_index(drop=True,inplace=True)
df

---
# **4. Duplicate Values (Column-Level)**
### **Identify  Duplicated Values in a Column**

In [ ]:
df['Email'].duplicated()

### **Get Rows Containing Duplicate Values**


In [ ]:
df[df['Email'].duplicated()]

### **Count Frequency of Duplicates**

In [ ]:
df['Email'].value_counts()
df['Email'].value_counts().loc[lambda x:x >1]

---
# **5.  Best Techniques (REAL WORLD)**

In [24]:



user_activity = pd.DataFrame({
    'UserID': [1, 1, 1, 2, 2, 3, 4, 4],
    'Email': [
        'ali@gmail.com',
        'ali@gmail.com',
        'ali@gmail.com',   # same user, multiple updates
        'sara@gmail.com',
        'sara@gmail.com',
        'ahmed@gmail.com',
        'bilal@gmail.com',
        'bilal@gmail.com'  # duplicate email
    ],
    'PageViews': [5, 7, 10, 3, 6, 4, 8, 9],
    'PurchaseAmount': [200, 0, 300, 150, 200, 0, 100, 250],
    'Visits': [1, 2, 3, 1, 2, 1, 2, 3],
    'updated_at': pd.to_datetime([
        '2024-01-01',
        '2024-01-10',
        '2024-02-01',   # latest record for User 1
        '2024-01-05',
        '2024-01-20',   # latest record for User 2
        '2024-01-15',
        '2024-01-18',
        '2024-02-05'    # latest record for User 4
    ])
})

user_activity


,UserID,Email,PageViews,PurchaseAmount,Visits,updated_at
0,1,ali@gmail.com,5,200,1,2024-01-01
1,1,ali@gmail.com,7,0,2,2024-01-10
2,1,ali@gmail.com,10,300,3,2024-02-01
3,2,sara@gmail.com,3,150,1,2024-01-05
4,2,sara@gmail.com,6,200,2,2024-01-20
5,3,ahmed@gmail.com,4,0,1,2024-01-15
6,4,bilal@gmail.com,8,100,2,2024-01-18
7,4,bilal@gmail.com,9,250,3,2024-02-05


### **Technique 1: Keep Row with Latest Timestamp**

In [23]:
user_activity.sort_values(by='updated_at',inplace=True)
user_activity.drop_duplicates(subset='UserID', keep='last', inplace=True)
user_activity


,UserID,Email,PageViews,PurchaseAmount,Visits,updated_at
5,3,ahmed@gmail.com,4,0,1,2024-01-15
4,2,sara@gmail.com,6,200,2,2024-01-20
2,1,ali@gmail.com,10,300,3,2024-02-01
7,4,bilal@gmail.com,9,250,3,2024-02-05


### **Technique 2: Aggregate Instead of Drop**
🧠 Prefer aggregation when data is meaningful, not noise.

In [27]:
user_activity.groupby('UserID',as_index=False).agg({ 'PurchaseAmount':'sum','Visits':'max'})

,UserID,PurchaseAmount,Visits
0,1,500,3
1,2,350,2
2,3,0,1
3,4,350,3


### **Technique 3: Deduplicate Before Merge**

In [ ]:
# right_df = right_df.drop_duplicates(subset='ID')
# pd.merge(left_df, right_df, on='ID')


### **Technique 4: Flag Instead of Remove**

In [29]:
user_activity['is_duplicate']=user_activity.duplicated(subset='UserID',keep=False)
user_activity

,UserID,Email,PageViews,PurchaseAmount,Visits,updated_at,is_duplicate
0,1,ali@gmail.com,5,200,1,2024-01-01,True
1,1,ali@gmail.com,7,0,2,2024-01-10,True
2,1,ali@gmail.com,10,300,3,2024-02-01,True
3,2,sara@gmail.com,3,150,1,2024-01-05,True
4,2,sara@gmail.com,6,200,2,2024-01-20,True
5,3,ahmed@gmail.com,4,0,1,2024-01-15,False
6,4,bilal@gmail.com,8,100,2,2024-01-18,True
7,4,bilal@gmail.com,9,250,3,2024-02-05,True
